In [1]:
import pygame
import sys
from copy import deepcopy

# Initialize Pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 800, 800
ROWS, COLS = 8, 8
SQUARE_SIZE = WIDTH // COLS

# RGB Colors
RED = (255, 0, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)
GRAY = (128, 128, 128)

# Set up the display
WIN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Checkers')

# Piece class equivalent
def create_piece(row, col, color):
    return {"row": row, "col": col, "color": color, "king": False, "x": 0, "y": 0}

def calc_pos(piece):
    piece["x"] = SQUARE_SIZE * piece["col"] + SQUARE_SIZE // 2
    piece["y"] = SQUARE_SIZE * piece["row"] + SQUARE_SIZE // 2

def make_king(piece):
    piece["king"] = True

def draw_piece(win, piece):
    radius = SQUARE_SIZE // 2 - 15
    pygame.draw.circle(win, GRAY, (piece["x"], piece["y"]), radius + 2)
    pygame.draw.circle(win, piece["color"], (piece["x"], piece["y"]), radius)
    if piece["king"]:
        pygame.draw.circle(win, BLUE, (piece["x"], piece["y"]), radius // 2)

def move_piece(piece, row, col):
    piece["row"] = row
    piece["col"] = col
    calc_pos(piece)

# Board class equivalent
def create_board():
    board = []
    for row in range(ROWS):
        board.append([])
        for col in range(COLS):
            if col % 2 == ((row + 1) % 2):
                if row < 3:
                    piece = create_piece(row, col, WHITE)
                    calc_pos(piece)
                    board[row].append(piece)
                elif row > 4:
                    piece = create_piece(row, col, RED)
                    calc_pos(piece)
                    board[row].append(piece)
                else:
                    board[row].append(0)
            else:
                board[row].append(0)
    return board

def draw_board(win):
    win.fill(BLACK)
    for row in range(ROWS):
        for col in range(row % 2, COLS, 2):
            pygame.draw.rect(win, RED, (row * SQUARE_SIZE, col * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))

def draw(win, board):
    draw_board(win)
    for row in range(ROWS):
        for col in range(COLS):
            piece = board[row][col]
            if piece != 0:
                draw_piece(win, piece)

def move(board, piece, row, col):
    board[piece["row"]][piece["col"]], board[row][col] = board[row][col], board[piece["row"]][piece["col"]]
    move_piece(piece, row, col)
    if row == ROWS - 1 or row == 0:
        make_king(piece)

def get_piece(board, row, col):
    return board[row][col]

def remove(board, pieces):
    for piece in pieces:
        board[piece["row"]][piece["col"]] = 0

def get_all_pieces(board, color):
    pieces = []
    for row in board:
        for piece in row:
            if piece != 0 and piece["color"] == color:
                pieces.append(piece)
    return pieces

def get_valid_moves(board, piece):
    moves = {}
    left = piece["col"] - 1
    right = piece["col"] + 1
    row = piece["row"]

    if piece["color"] == RED or piece["king"]:
        moves.update(traverse_left(board, row - 1, max(row - 3, -1), -1, piece["color"], left))
        moves.update(traverse_right(board, row - 1, max(row - 3, -1), -1, piece["color"], right))
    if piece["color"] == WHITE or piece["king"]:
        moves.update(traverse_left(board, row + 1, min(row + 3, ROWS), 1, piece["color"], left))
        moves.update(traverse_right(board, row + 1, min(row + 3, ROWS), 1, piece["color"], right))

    return moves

def traverse_left(board, start, stop, step, color, left, skipped=[]):
    moves = {}
    last = []
    for r in range(start, stop, step):
        if left < 0:
            break

        current = board[r][left]
        if current == 0:
            if skipped and not last:
                break
            elif skipped:
                moves[(r, left)] = last + skipped
            else:
                moves[(r, left)] = last

            if last:
                if step == -1:
                    row = max(r - 3, 0)
                else:
                    row = min(r + 3, ROWS)
                moves.update(traverse_left(board, r + step, row, step, color, left - 1, skipped=last))
                moves.update(traverse_right(board, r + step, row, step, color, left + 1, skipped=last))
            break
        elif current["color"] == color:
            break
        else:
            last = [current]

        left -= 1

    return moves

def traverse_right(board, start, stop, step, color, right, skipped=[]):
    moves = {}
    last = []
    for r in range(start, stop, step):
        if right >= COLS:
            break

        current = board[r][right]
        if current == 0:
            if skipped and not last:
                break
            elif skipped:
                moves[(r, right)] = last + skipped
            else:
                moves[(r, right)] = last

            if last:
                if step == -1:
                    row = max(r - 3, 0)
                else:
                    row = min(r + 3, ROWS)
                moves.update(traverse_left(board, r + step, row, step, color, right - 1, skipped=last))
                moves.update(traverse_right(board, r + step, row, step, color, right + 1, skipped=last))
            break
        elif current["color"] == color:
            break
        else:
            last = [current]

        right += 1

    return moves

def minimax(board, depth, max_player, game):
    if depth == 0 or winner(board) is not None:
        return evaluate(board), board

    if max_player:
        max_eval = float('-inf')
        best_move = None
        for move in get_all_moves(board, WHITE, game):
            evaluation = minimax(move, depth - 1, False, game)[0]
            max_eval = max(max_eval, evaluation)
            if max_eval == evaluation:
                best_move = move
        return max_eval, best_move
    else:
        min_eval = float('inf')
        best_move = None
        for move in get_all_moves(board, RED, game):
            evaluation = minimax(move, depth - 1, True, game)[0]
            min_eval = min(min_eval, evaluation)
            if min_eval == evaluation:
                best_move = move
        return min_eval, best_move

def evaluate(board):
    red_pieces = white_pieces = red_kings = white_kings = 0
    for row in board:
        for piece in row:
            if piece != 0:
                if piece["color"] == RED:
                    red_pieces += 1
                    if piece["king"]:
                        red_kings += 1
                else:
                    white_pieces += 1
                    if piece["king"]:
                        white_kings += 1
    return white_pieces - red_pieces + (white_kings * 0.5 - red_kings * 0.5)

def get_all_moves(board, color, game):
    moves = []
    for piece in get_all_pieces(board, color):
        valid_moves = get_valid_moves(board, piece)
        for move, skip in valid_moves.items():
            temp_board = deepcopy(board)
            temp_piece = temp_board[piece["row"]][piece["col"]]
            new_board = simulate_move(temp_piece, move, temp_board, skip, game)
            moves.append(new_board)
    return moves

def simulate_move(piece, move, board, skip, game):
    move_piece(piece, move[0], move[1])
    if skip:
        remove(board, skip)
    return board

def winner(board):
    red_left = white_left = 0
    for row in board:
        for piece in row:
            if piece != 0:
                if piece["color"] == RED:
                    red_left += 1
                else:
                    white_left += 1
    if red_left <= 0:
        return WHITE
    elif white_left <= 0:
        return RED
    return None

def draw_valid_moves(win, moves):
    for move in moves:
        row, col = move
        pygame.draw.circle(win, BLUE, (col * SQUARE_SIZE + SQUARE_SIZE // 2, row * SQUARE_SIZE + SQUARE_SIZE // 2), 15)

def main():
    run = True
    clock = pygame.time.Clock()
    board = create_board()
    turn = RED
    selected = None
    valid_moves = {}

    while run:
        clock.tick(60)

        if turn == WHITE:
            value, new_board = minimax(board, 3, True, None)
            board = new_board
            turn = RED

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False

            if event.type == pygame.MOUSEBUTTONDOWN:
                pos = pygame.mouse.get_pos()
                row, col = pos[1] // SQUARE_SIZE, pos[0] // SQUARE_SIZE
                if selected:
                    result = move(board, selected, row, col)
                    if not result:
                        selected = None
                        piece = get_piece(board, row, col)
                        if piece != 0 and piece["color"] == turn:
                            selected = piece
                            valid_moves = get_valid_moves(board, piece)
                    else:
                        skipped = valid_moves[(row, col)]
                        if skipped:
                            remove(board, skipped)
                        turn = WHITE if turn == RED else RED
                        selected = None
                        valid_moves = {}
                else:
                    piece = get_piece(board, row, col)
                    if piece != 0 and piece["color"] == turn:
                        selected = piece
                        valid_moves = get_valid_moves(board, piece)

        draw(WIN, board)
        if selected:
            draw_valid_moves(WIN, valid_moves)
        pygame.display.update()

    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()


pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\user\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
